In [1]:
import pandas as pd
cleaned_data = pd.read_csv(r'L:\Guvi\Project 4\cleaned_data.csv')
encoded_data = pd.read_csv(r'L:\Guvi\Project 4\encoded_data.csv')

In [4]:
encoded_data = encoded_data.select_dtypes(include=['number'])



In [5]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity
similarity_matrix = cosine_similarity(encoded_data)

# Recommendation function
def recommend_restaurants_cosine(restaurant_id, top_n=5):
    # Find the index of the restaurant
    idx = cleaned_data[cleaned_data['id'] == restaurant_id].index[0]

    # Get the similarity scores
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # Sort the restaurants by similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of top_n similar restaurants (skip the first one because it's the restaurant itself)
    sim_indices = [i[0] for i in sim_scores[1:top_n+1]]

    # Return the recommended restaurants
    return cleaned_data.iloc[sim_indices][['id', 'name', 'city', 'rating', 'cost', 'cuisine']]


In [6]:
from sklearn.cluster import KMeans

# Build KMeans model
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(encoded_data)

# Assign cluster labels
cleaned_data['cluster'] = kmeans.labels_

# Recommendation function
def recommend_restaurants_kmeans(restaurant_id, top_n=5):
    # Find the cluster of the selected restaurant
    idx = cleaned_data[cleaned_data['id'] == restaurant_id].index[0]
    cluster_label = cleaned_data.iloc[idx]['cluster']

    # Find all restaurants in the same cluster
    cluster_restaurants = cleaned_data[cleaned_data['cluster'] == cluster_label]

    # Exclude the input restaurant
    cluster_restaurants = cluster_restaurants[cluster_restaurants['id'] != restaurant_id]

    # Return top_n restaurants
    return cluster_restaurants[['id', 'name', 'city', 'rating', 'cost', 'cuisine']].head(top_n)
